<img src='../../media/common/LogoWekeo_Copernicus_RGB_0.png' align='left' height='96px'></img>

<hr>

# Analyzing Wet Snow Extent from SWS Time Series Based on Altitude

## Environment Setup
Before we begin, we need to prepare our environment by installing and importing the necessary R packages.

### Load Required Libraries

In [43]:
library(terra)
library(fs)
library(ggplot2)

library(magrittr)
library(dplyr)
library(scales)

## Functions

#### Function to Open and Read GeoTIFF Files, Assuming Downloaded Products Have Already Been Extracted

In [3]:
read_tif <- function(file_tif) {
  ds <- rast(file_tif)

  print(dim(ds))
  
  proj <- crs(ds)
  width <- ncol(ds)
  height <- nrow(ds)
  count <- nlyr(ds)
  meta <- list(
    min = minmax(ds)[1, ],
    max = minmax(ds)[2, ],
    mean = global(ds, fun = mean, na.rm = TRUE),
    sd = global(ds, fun = sd, na.rm = TRUE)
  )
  
  list(data = ds, width = width, height = height, count = count, meta = meta, proj = proj)
}

## Usage

<font color='red'>Enter a valid path and file names into the following box. These are mandatory as input for all following steps:</font>
***

#### <font color='#0080FF'>"path_input"</font>: Enter local path on your machine where products are stored. 
<table><tr><td>
Note: If you copy/paste the path from a windows explorer, add an <b>"r"</b> before the string to interpret backslashes as slashes, as shown in the example below. If your path contains already slashes, remove the <b>"r"</b> in the beginning of the string.
</td></tr></table>

#### <font color='#0080FF'> "files_wsm"</font>: Define the wet snow product file names to be loaded. 
<table><tr><td>
Note: In the template, all files starting with <b>'SWS_'</b> and ending with <b>'_WSM.tif'</b> are searched.
</td></tr></table>

#### <font color='#0080FF'> "file_elev"</font>: Define the digital elevation model file name with the same geometry as the SWS products to be used for the analysis.

<table><tr><td>
Note: In this template, the DEM file name must include <b>"DEM_60m"</b> and extension should be <b>".tif"</b>, and must be stored in the same directory as the SWS products.
</td></tr></table>

In [4]:
path_input <- normalizePath("../../data/download/snow-and-ice/SWS/")

files_wsm <- list.files(path = path_input, pattern = "SWS_.*_WSM\\.tif$", full.names = TRUE)
file_elev <- list.files(path = path_input, pattern = ".*DEM_60m.*\\.tif$", full.names = TRUE)

First, open and read all Wet Snow products for the mountains using the function defined in Box [2].

In [ ]:
wsm_datas <- list()

for (file_wsm in files_wsm) {
  wsm_datas <- append(wsm_datas, list(read_tif(file_wsm)$data))
}

Next, open and read the elevation data using the same function from Box [2].

In [ ]:
elev_data <- read_tif(file_elev[1])$data
elev_data[is.na(elev_data)] <- 255

After loading the data, plot the elevation data and its associated colorbar for visualization.

In [ ]:
elev_df <- as.data.frame(elev_data, xy = TRUE) %>% setNames(c("x", "y", "value"))
#print(head(elev_df))

options(repr.plot.width = 8, repr.plot.height = 6)
ggplot(elev_df, aes(x = x, y = y, fill = value)) +
  geom_raster(interpolate = FALSE) + 
  coord_fixed() +
  theme_void() +
  theme(
    legend.position = "right",
    legend.direction = "vertical",
    legend.justification = "center",
    legend.key.height = unit(2, "cm"),
    legend.box.margin = unit(c(0, 2, 2, 2), "cm"),
    legend.text = element_text(size = 16),
    legend.title = element_blank()
  ) +
  labs(fill = "Elevation")

Then, plot the first SWS product of the time series, ensuring to add a standard colorbar (not the one attached to the product)

For details on the SWS product coding, please refer to the Product User Manual (https://land.copernicus.eu/user-corner/technical-library/hrsi-snow-pum):

In [ ]:
tab20b_colors <- c(
  "#393b79", "#5254a3", "#6b6ecf", "#9c9ede",
  "#637939", "#8ca252", "#b5cf6b", "#cedb9c",
  "#8c6d31", "#bd9e39", "#e7ba52", "#e7cb94",
  "#843c39", "#ad494a", "#d6616b", "#e7969c",
  "#7b4173", "#a55194", "#ce6dbd", "#de9ed6"
)

wsm_data <- wsm_datas[[1]]
wsm_data[is.na(wsm_data)] <- 255

wsm_df <- as.data.frame(wsm_data, xy = TRUE) %>% setNames(c("x", "y", "value"))

options(repr.plot.width = 8, repr.plot.height = 6)
ggplot(wsm_df, aes(x = x, y = y, fill = value)) +
  geom_raster(interpolate = FALSE) + 
   scale_fill_gradientn(colours = tab20b_colors, values = rescale(1:20)) +
  coord_fixed() +
  theme_void() +
  theme(
    legend.position = "right",
    legend.direction = "vertical",
    legend.justification = "center",
    legend.key.height = unit(2, "cm"),
    legend.box.margin = unit(c(0, 2, 2, 2), "cm"),
    legend.text = element_text(size = 16),
    legend.title = element_blank()
  ) +
  labs(fill = "Elevation")


Now, define the elevation intervals to be used for the analysis.
<table><tr><td>
Note: in this example, the elevation range covers 1000 - 4000 m a.s.l., using an altitude interval of 200 m.
</td></tr></table>

In [38]:
dbin <- 200
bins <- seq(1000, 4000 + dbin, by = dbin)
elevs <- bins[1:(length(bins) - 1)] + dbin / 2

Extract the elevation information for wet snow pixels and add this data to a database for all SWS products in the time series.

In [ ]:
val_wet_snow <- 110

hists <- list()
for (wsm_data in wsm_datas) {
  sel <- (values(wsm_data) == val_wet_snow)
  
  elev_sel <- values(elev_data)[sel]
  
  hists[[length(hists) + 1]] <- hist(elev_sel, breaks = bins, plot = FALSE)$counts
}

Retrieve the dates from the SWS product file names to be used as legends in the next step:

In [40]:
labels_wsm <- sapply(files_wsm, function(ele) {
  tools::file_path_sans_ext(basename(ele))
})

Finally, prepare a plot showing the wet snow extent in relation to the altitude for the full SWS time series used as input.

Ensure that the legend (from the previous step) and axis labels are added for clarity.

In [ ]:
# Prepare the data for plotting
plot_data <- data.frame()
for (i in seq_along(hists)) {
  plot_data <- rbind(plot_data, data.frame(
    hist = hists[[i]],
    elevs = elevs,
    label_wsm = substr(labels_wsm[i], 5, 12)
  ))
}

# Plot the data
ggplot(plot_data, aes(x = hist, y = elevs, color = label_wsm)) +
  geom_line() +
  labs(
    title = "",
    x = "Number of pixels",
    y = "Altitude [m a.s.l.]",
    color = "WSM Label"
  ) +
  theme_minimal() +
  theme(legend.position = "right")


***
><span style = "font-family:Verdana; font-size:0.7em">Copyright © <font color='darkblue'>2022</font>, by ENVEO IT GmbH.</span>  
<span style = "font-family:Verdana; font-size:0.7em">Contributors: Lars Keuris,  Gabriele Schwaizer</span>  
<span style = "font-family:Verdana; font-size:0.7em">URL: www.enveo.at</span> 
***
<p style = "font-family:Verdana; font-size:0.7em; line-height:0.5">Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files, to use the Software without restriction.</p>  
<p style = "font-family:Verdana; font-size:0.7em; line-height:1.15; text-align:justify">THE SOFTWARE IS PROVIDED AS IS, WITHOUT WARRANTY OF ANYKIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. <b>IN NO EVENT SHALL THE COPYRIGHT HOLDER BE LIABLE FOR ANY CLAIM, DIRECT INDIRECT, SPECIAL, INCIDENTAL, OR CONSEQUENTIAL DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</b></p>

*** 